## 3. Логистическая регрессия

Обучите модель логистической регрессии. В качестве предикторов выступают поля: __chocolate, fruity, caramel, peanutyalmondy, nougat, crispedricewafer, hard, bar, pluribus, sugarpercent, pricepercent__, отклик — Y.

В качестве тренировочного набора данных используйте данные из файла, за иключением следующих конфет: *Laffy Taffy, Nestle Crunch, Snickers*. Обучите модель.

* Если используете Azure ML Studio, укажите random seed = 2019 в параметрах блока Logistic Regression.
* Если используете Python, используйте модель с параметрами: LogisticRegression(random_state=2019, solver='lbfgs').

Обучите модель и выполните предсказание для всех конфет из прилагаемого файла тестовых данных.

In [1]:
#чтение данных, так как названия конфет уникальны, используем их в качестве индекса
import pandas as pd
import numpy as np
DATA = pd.read_csv("candy-data.csv", delimiter=',', index_col='competitorname')
DATA.head()

,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent,Y
competitorname,,,,,,,,,,,,,
100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725,1
3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936,1
One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086,0
One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505,0
Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465,1


In [2]:
#обучение модели будем проводить на данных, за исключением некоторых конфет
train_data = DATA.drop(['Laffy Taffy', 'Nestle Crunch', 'Snickers'])
#отбор данных для предикторов, удаление дввух последних столбцов, индекс не включается в данные автоматически.
X = pd.DataFrame(train_data.drop(['winpercent', 'Y'], axis=1))
#отбор столбца для отклика
y = pd.DataFrame(train_data['Y'])

In [3]:
DATA_test = pd.read_csv("candy-test.csv", delimiter=',', index_col='competitorname')
X_test = pd.DataFrame(DATA_test.drop(['Y'], axis=1))
Y_true = (DATA_test['Y'].to_frame().T).values.ravel()
DATA_test.head()

,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,Y
competitorname,,,,,,,,,,,,
Sugar Babies,0,1,1,0,0,0,1,0,1,0.647364,0.7670,0
Sugar Daddy,1,0,0,0,1,0,1,0,0,0.418000,0.3250,0
Super Bubble,0,1,0,0,0,0,0,0,0,0.162000,0.1160,0
Swedish Fish,0,1,0,0,0,0,0,0,1,0.604000,0.7550,1
Tootsie Pop,1,0,0,0,0,0,0,0,0,0.876560,0.5654,1


#### Обучение модели логистической регрессии

In [4]:
#подключение модели логистической регрессии из библиотеки sklearn
from sklearn.linear_model import LogisticRegression
#обучение модели
model = LogisticRegression(random_state=2019, solver='lbfgs').fit(X, y.values.ravel())

#### Введите вероятность отнесения конфеты Sugar Daddy к классу 1:
Десятичный разделитель точка. Введите ответ с точностью до трех знаков.

In [5]:
#отбор данных для предикторов и предсказание
np.round(model.predict_proba(DATA_test.loc['Sugar Daddy',:].to_frame().T.drop(['Y'], axis=1))[0][1], 3)

0.35

#### Введите вероятность отнесения конфеты Tootsie Roll Midgies к классу 1:
Десятичный разделитель точка. Введите ответ с точностью до трех знаков

In [6]:
#отбор данных для предикторов и предсказание
np.round(model.predict_proba(DATA_test.loc['Tootsie Roll Midgies',:].to_frame().T.drop(['Y'], axis=1))[0][1], 3)

0.666

### Выполните оценку модели с помощью матрицы ошибок и рассчитайте следующие параметры при пороге отсечения (Treshhold) = 0.5.

In [7]:
#подключение библиотеки для вычисления метрик
from sklearn import metrics
#предсказываем
Y_pred = model.predict(X_test)

#### Введите значение Recall, или TPR для тестового набора данных:
Десятичный разделитель точка. Введите ответ с точностью до трех знаков.

In [8]:
#вычисление Recall
metrics.recall_score(Y_true, Y_pred)

0.6

#### Введите значение Precision для тестового набора данных:
Десятичный разделитель точка. Введите ответ с точностью до трех знаков.

In [9]:
#вычисление Precision
metrics.precision_score(Y_true, Y_pred)

0.5

#### Введите значение AUC для тестового набора данных:
Десятичный разделитель точка. Введите ответ с точностью до трех знаков.

In [10]:
#вычисление FPR и TPR, с последующим нахождением AUC
fpr, tpr, thresholds = metrics.roc_curve(Y_true, Y_pred)
np.round(metrics.auc(fpr, tpr), 3)

0.65